In [1]:
import sys
sys.version

'3.7.5 (default, Oct 30 2019, 04:53:00) \n[GCC 6.3.0 20170516]'

In [ ]:
from multiprocessing import Pool
import multiprocessing
pool = Pool(4)
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
start = time.time()
# data_chunks = list(zip(select_frames, mel_chunks, select_face_results))
# data_chunks = list(chunks(data_chunks,16))
select_frames = list(chunks(select_frames,16))
mel_chunks = list(chunks(mel_chunks,16))
select_face_results = list(chunks(select_face_results,16))
data_chunks = list(zip(select_frames, mel_chunks, select_face_results))
print(time.time()-start)
gen = pool.imap(datagen_parallel,data_chunks)
print(time.time()-start)

In [ ]:
start = time.time()
gen = list(datagen(select_frames, mel_chunks, select_face_results, 16))
print("Gen:", time.time() - start)
len(gen)

In [ ]:
start = time.time()

gen = list(datagen(select_frames, mel_chunks, select_face_results, 16))

print("Gen:", time.time() - start)

final_frames = []

start = time.time()
for i, (img_batch, mel_batch, frames, coords) in enumerate(gen):
    
    img_batch = torch.FloatTensor(np.transpose(img_batch, (0, 3, 1, 2))).to(device)
    mel_batch = torch.FloatTensor(np.transpose(mel_batch, (0, 3, 1, 2))).to(device)
    
    with torch.no_grad():
        pred = model(mel_batch, img_batch)
    
    pred = pred.cpu().numpy().transpose(0, 2, 3, 1) * 255.

    print(time.time() - start)
    
    for p, f, c in zip(pred, frames, coords):
        y1, y2, x1, x2 = c
        p = cv2.resize(p.astype(np.uint8), (x2 - x1, y2 - y1))
        f[y1:y2, x1:x2] = p
        final_frames.append(f)
    print(time.time() - start)

In [ ]:
0.006945371627807617 * 16

In [ ]:
frame_h, frame_w = select_frames[0].shape[:-1]
out = cv2.VideoWriter('temp/result.avi', cv2.VideoWriter_fourcc(*'DIVX'), fps, (frame_w, frame_h))
for f in final_frames:
    out.write(f)
out.release()
command = 'ffmpeg -y -i {} -i {} -strict -2 -q:v 1 {}'.format(args.audio, 'temp/result.avi', args.outfile)
subprocess.call(command, shell=platform.system() != 'Windows')
print("combine video & audio: ", time.time() - start)
print(time.time() - start)

In [ ]:
f.shape